## Preparation

In [ ]:
# import library
import requests
import pandas as pd
import datetime
from urllib import parse
from bs4 import BeautifulSoup
import time
import json
import re
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Scripts for collecting by API

In [ ]:
current_directory = '/content/drive/MyDrive/Colab Notebooks/big data/keiba_ver2'

In [ ]:
course_region = {'京都':'61286', '東京':'44116', '盛岡':'33431', '大井':'44132', '中山':'45106', '中京':'51216', '船橋':'45106', '金沢':'56227', '浦和':'43241', '新潟':'54232', '佐賀':'82306', '札幌':'14163', '門別':'22141', '小倉':'82056', '函館':'23232', '福島':'36127', '川崎':'44166', '名古屋':'51216', '園田':'62051', '阪神':'62051', '高知':'74182', '水沢':'33716'}

In [ ]:
def update_temperature_data(region_name, station_id, csv_file_path):
    # CSV 파일을 읽어오기
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path)

        # 파일이 비어있는지 확인
        if df.empty:
            print(f"{region_name}: CSV 파일이 비어 있습니다. 데이터 수집을 중지합니다.")
            return  # 함수 종료

        last_date = pd.to_datetime(df['年月日']).max()  # 마지막 날짜
        print(f"{region_name} CSV 파일에서 마지막 날짜: {last_date}")
    else:
        print(f"{region_name}: CSV 파일이 존재하지 않습니다. 데이터 수집을 중지합니다.")
        return  # 함수 종료

    # 오늘 날짜 전날까지 수집할 날짜 설정
    today = datetime.date.today()
    start_date = last_date.date() + datetime.timedelta(days=1)  # 마지막 날짜를 datetime.date로 변환
    end_date = today - datetime.timedelta(days=1)

    if start_date > end_date:
        print(f"{region_name}: 추가할 데이터가 없습니다.")
        return  # 함수 종료

    # 수집한 데이터를 저장할 리스트
    new_data = []

    # 날짜 범위 동안의 데이터를 수집
    current_date = start_date
    while current_date <= end_date:
        yyyymmdd = current_date.strftime("%Y%m%d")
        amedas_url = f"https://www.jma.go.jp/bosai/amedas/data/point/{station_id}/{yyyymmdd}_00.json"

        try:
            amedas_req = requests.get(amedas_url)
            amedas_data = amedas_req.json()

            # 하루 동안의 기온 데이터를 수집하고 평균 계산
            temperatures = []
            for hour_data in amedas_data.values():
                if 'temp' in hour_data and hour_data['temp'][0] is not None:
                    temperatures.append(hour_data['temp'][0])

            if temperatures:
                average_temperature = sum(temperatures) / len(temperatures)
                # 소수점 1자리까지 반올림
                rounded_temperature = round(average_temperature, 1)
                new_data.append({
                    "年月日": current_date.strftime("%Y-%m-%d"),
                    "平均気温(℃)": rounded_temperature
                })
            else:
                print(f"{current_date}에 기온 데이터가 없습니다.")

        except requests.exceptions.JSONDecodeError:
            print(f"{current_date}에 데이터를 불러오지 못했습니다.")

        # 날짜를 하루씩 증가
        current_date += datetime.timedelta(days=1)

    # 수집한 데이터를 기존 CSV 파일에 추가
    if new_data:
        new_df = pd.DataFrame(new_data)
        df = pd.concat([df, new_df], ignore_index=True)
        df.to_csv(csv_file_path, index=False)
        print(f"{region_name}: 데이터를 업데이트하였습니다.")
    else:
        print(f"{region_name}: 수집된 데이터가 없습니다.")

In [ ]:
# 각 지역에 대해 데이터를 업데이트
def run_update_temperature_data():
    for region, station_id in course_region.items():
        csv_file_path = f"{current_directory}/気温/{region}/{region}.csv"  # 각 지역에 맞는 파일명
        update_temperature_data(region, station_id, csv_file_path)

        return 'finish!'

## Start Collecting

In [ ]:
run_update_temperature_data()